In [8]:
!pip install tvdatafeed
!pip install pandas_ta

!ngrok authtoken 2Ffv1unEPZYVnPjuDvjxUXVlh0y_RHo2BDqLMkLf8CPAqJeD
!ngrok config add-authtoken 2Ffv1unEPZYVnPjuDvjxUXVlh0y_RHo2BDqLMkLf8CPAqJeD

!pip install streamlit
!pip install pyngrok

from pyngrok import ngrok

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml
NAME:
   ngrok - tunnel local ports to public URLs and inspect traffic

DESCRIPTION:
    ngrok exposes local networked services behinds NATs and firewalls to the
    public internet over a secure tunnel. Share local websites, build/test
    webhook consumers and self-host personal services.
    Detailed help for each command is available with 'ngrok help <command>'.
    Open http://localhost:4040 for ngrok's web interface to inspect traffic.

EXAMPLES:
    ngrok http 80                    # secure public URL for port 80 web server
    ngrok http -subdomain=baz 8080   # port 8080 available at baz.ngrok.io
    ngrok http foo.dev:80            # tunnel to host:port instead of localhost
    ngrok http https://localhost     # expose a

In [2]:
%%writefile azreal.py

def FEMUR():
    from tvDatafeed import TvDatafeed, Interval
    import numpy as np
    import pandas as pd
    import pandas_ta as ta

    Forex_Pairs_List = ["EURUSD","USDJPY","GBPUSD","AUDUSD","USDCHF","NZDUSD","USDCAD",
                       "EURJPY","EURGBP","EURAUD","EURCHF","EURNZD","EURCAD",
                       "GBPJPY","CHFJPY","NZDJPY","AUDJPY","CADJPY",
                       "GBPAUD","AUDCHF","AUDNZD","AUDCAD",
                       "GBPCHF","NZDCHF","CADCHF",
                       "GBPNZD","GBPCAD",
                       "NZDCAD"]

    Final_df = pd.DataFrame()

    username = 'Azreal1'
    password = 'Mynameisdhruv123!@#'

    tv = TvDatafeed(username, password)

    for Currency_Pair in Forex_Pairs_List:
        Symbol_String = Currency_Pair
        Currency_Pair = tv.get_hist(symbol = 'FX:{}'.format(Currency_Pair), exchange = 'FXCM',
                                    interval = Interval.in_1_hour, n_bars = 100)
        #Stochastic
        Stoch = round(ta.stoch(high = Currency_Pair["high"], low = Currency_Pair["low"], 
                               close = Currency_Pair["close"], window = 14, smooth_window = 3),2)
        Currency_Pair["Stochastic %K"] = Stoch["STOCHk_14_3_3"]
        Currency_Pair["Stochastic %D"] = Stoch["STOCHd_14_3_3"]
        #Heiken Ashi
        if Symbol_String[-3:] == "JPY":
            Rounding = 3
        else:
            Rounding = 5
        Heiken_Ashi = round(ta.ha(Currency_Pair["open"], high = Currency_Pair["high"], 
                                  low = Currency_Pair["low"], close = Currency_Pair["close"]),Rounding)
        Currency_Pair["Heiken Ashi Open"] = Heiken_Ashi["HA_open"]
        Currency_Pair["Heiken Ashi High"] = Heiken_Ashi["HA_high"]
        Currency_Pair["Heiken Ashi Low"] = Heiken_Ashi["HA_low"]
        Currency_Pair["Heiken Ashi Close"] = Heiken_Ashi["HA_close"]

        #Heiken Ashi Bool
        Heiken_Ashi_Boolean = []

        i = 0
        while i < Currency_Pair.shape[0]:
            if (Currency_Pair["Heiken Ashi Close"][i] - Currency_Pair["Heiken Ashi Open"][i]) >= 0:
                Heiken_Ashi_Boolean.append(1)
            elif(Currency_Pair["Heiken Ashi Close"][i] - Currency_Pair["Heiken Ashi Open"][i]) < 0:
                Heiken_Ashi_Boolean.append(0)
            i+=1
        Currency_Pair["Heiken Ashi Boolean"] = Heiken_Ashi_Boolean

        # Peak swing high and low calculations
        Peak_Value = []
        Peak_Stochastic_Value = []
        Peak_Value_List = list()
        Peak_Stochastic_Value_List = list()
        Peak_Value_List = [np.nan]*Currency_Pair.shape[0]
        Peak_Stochastic_Value_List = [np.nan]*Currency_Pair.shape[0]

        i = 0

        while i < Currency_Pair.shape[0] - 1:
            if Currency_Pair["Heiken Ashi Boolean"][i] == 1 and Currency_Pair["Heiken Ashi Boolean"][i+1] == 0:
                Peak_Value.append(Currency_Pair["close"][i])
                Peak_Stochastic_Value.append(Currency_Pair["Stochastic %K"][i])
                j = i - 1
                while Currency_Pair["Heiken Ashi Boolean"][j] == 1:
                    Peak_Value.append(Currency_Pair["close"][j])
                    Peak_Stochastic_Value.append(Currency_Pair["Stochastic %K"][j])
                    j-=1
                Max_Value = max(Peak_Value)
                Max_Stochastic_Value = max(Peak_Stochastic_Value)
                Peak_Value = []
                Peak_Stochastic_Value = []
                Peak_Value_List[i+1] = Max_Value
                Peak_Stochastic_Value_List[i+1] = Max_Stochastic_Value
                i+=1
            elif Currency_Pair["Heiken Ashi Boolean"][i] == 0 and Currency_Pair["Heiken Ashi Boolean"][i+1] == 1:
                Peak_Value.append(Currency_Pair["close"][i])
                Peak_Stochastic_Value.append(Currency_Pair["Stochastic %K"][i])
                j = i - 1
                while Currency_Pair["Heiken Ashi Boolean"][j] == 0:
                    Peak_Value.append(Currency_Pair["close"][j])
                    Peak_Stochastic_Value.append(Currency_Pair["Stochastic %K"][j])
                    j-=1
                Min_Value = min(Peak_Value)
                Min_Stochastic_Value = min(Peak_Stochastic_Value)
                Peak_Value = []
                Peak_Stochastic_Value = []
                Peak_Value_List[i+1] = Min_Value
                Peak_Stochastic_Value_List[i+1] = Min_Stochastic_Value
                i+=1
            elif Currency_Pair["Heiken Ashi Boolean"][i] == 1 and Currency_Pair["Heiken Ashi Boolean"][i+1] == 1:
                i+=1
            elif Currency_Pair["Heiken Ashi Boolean"][i] == 0 and Currency_Pair["Heiken Ashi Boolean"][i+1] == 0:
                i+=1

        Currency_Pair["Peak Value"] = Peak_Value_List
        Currency_Pair["Stochastic Peak Value"] = Peak_Stochastic_Value_List

        Swing_High_Recent = np.nan
        Stochastic_High_Recent = np.nan
        Swing_Low_Recent = np.nan
        Stochastic_Low_Recent = np.nan

        Looking_For_Shorts = np.nan
        Looking_For_Longs = np.nan

        Divergence_List = list()
        Divergence_List = [np.nan]*Currency_Pair.shape[0]

        i = 0

        while i < Currency_Pair.shape[0] - 1:
            if (pd.isna(Currency_Pair["Peak Value"][i]) == False and Currency_Pair["Heiken Ashi Boolean"][i] == 1):
                Swing_Low_Recent = Currency_Pair["Peak Value"][i]
                Stochastic_Low_Recent = Currency_Pair["Stochastic Peak Value"][i]
                Looking_For_Shorts = True
                Looking_For_Longs = False
                i+=1
            elif (pd.isna(Currency_Pair["Peak Value"][i]) == False and Currency_Pair["Heiken Ashi Boolean"][i] == 0):
                Swing_High_Recent = Currency_Pair["Peak Value"][i]
                Stochastic_High_Recent = Currency_Pair["Stochastic Peak Value"][i]
                Looking_For_Shorts = False
                Looking_For_Longs = True
                i+=1
            if (pd.isna(Swing_High_Recent) == False and pd.isna(Swing_Low_Recent) == False and Looking_For_Shorts == True and
                Currency_Pair["close"][i] >= Swing_High_Recent and Currency_Pair["Stochastic %K"][i] <= Stochastic_High_Recent):
                Divergence_List[i] = "Divergence Short"
                i+=1
            elif (pd.isna(Swing_High_Recent) == False and pd.isna(Swing_Low_Recent) == False and Looking_For_Longs == True and
                Currency_Pair["close"][i] <= Swing_Low_Recent and Currency_Pair["Stochastic %K"][i] >= Stochastic_Low_Recent):
                Divergence_List[i] = "Divergence Long"
                i+=1
            else:
                i+=1

        Currency_Pair["Divergence"] = Divergence_List

        Final_df = pd.concat([Final_df , Currency_Pair.iloc[[-2]]])
    return Final_df
    
import streamlit as st
st.title("Welcome to Azreal")

if st.button("Generate Forex Matrix"):
  Output = FEMUR()
  st.dataframe(Output)


Writing azreal.py


In [11]:
public_url = ngrok.connect(port=80)
public_url

<NgrokTunnel: "http://623a-35-229-66-192.ngrok.io" -> "http://localhost:80">

In [12]:
!streamlit run azreal.py --server.port=80

2022-10-05 16:30:30.759 INFO    numexpr.utils: NumExpr defaulting to 2 threads.



  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.2:80
  External URL: http://35.229.66.192:80

  Stopping...
^C


In [13]:
!pgrep streamlit
!kill 445
ngrok.kill()

/bin/bash: line 0: kill: (445) - No such process
